# <h1 align=center> **MACHINE LEARNING** </h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import warnings
from surprise import Dataset,Reader,SVD,accuracy
from surprise.model_selection import train_test_split
import pyarrow.parquet as pq
import pickle 
warnings.filterwarnings("ignore")  # Suprimir todos los warnings
# Instanciamos los dataset
games=pd.read_parquet("games.parquet")
reviews=pd.read_parquet("reviews.parquet")

Combinamos mediante un Merge los DataFrames 'reviews' y 'games' por item_id

In [2]:
merged_df = pd.merge(reviews[['item_id', 'user_id', 'sentiment_analysis',"recommend"]], games[['app_name',"item_id"]], left_on='item_id', right_on="item_id")
merged_df

,item_id,user_id,sentiment_analysis,recommend,app_name
0,1250,76561197970982479,2,True,Killing Floor
1,22200,76561197970982479,2,True,Zeno Clash
2,227300,js41637,2,True,Euro Truck Simulator 2
3,239030,js41637,2,True,"Papers, Please"
4,370360,evcentric,2,True,TIS-100
...,...,...,...,...,...
44734,730,76561198239215706,2,True,Counter-Strike: Global Offensive
44735,730,wayfeng,1,True,Counter-Strike: Global Offensive
44736,253980,76561198251004808,2,True,Enclave
44737,730,72947282842,0,True,Counter-Strike: Global Offensive


Trasnfomamos el valor booleano de la columna 'recommend' por numerico para el calculo del rating

In [3]:
merged_df['recommend'] = merged_df['recommend'].replace({True: 1, False: 0})
merged_df

,item_id,user_id,sentiment_analysis,recommend,app_name
0,1250,76561197970982479,2,1,Killing Floor
1,22200,76561197970982479,2,1,Zeno Clash
2,227300,js41637,2,1,Euro Truck Simulator 2
3,239030,js41637,2,1,"Papers, Please"
4,370360,evcentric,2,1,TIS-100
...,...,...,...,...,...
44734,730,76561198239215706,2,1,Counter-Strike: Global Offensive
44735,730,wayfeng,1,1,Counter-Strike: Global Offensive
44736,253980,76561198251004808,2,1,Enclave
44737,730,72947282842,0,1,Counter-Strike: Global Offensive


Contruimos un rating constrastando las recomendaciones con el resultado del analisis del sentimientos 

In [4]:
import numpy as np
scala = [5, 4, 3, 2, 1, 0]
merged_df['rating'] = np.select(
    [
        (merged_df['sentiment_analysis'] == 2) & (merged_df['recommend'] == 1),
        (merged_df['sentiment_analysis'] == 2) & (merged_df['recommend'] == 0),
        (merged_df['sentiment_analysis'] == 1) & (merged_df['recommend'] == 1),
        (merged_df['sentiment_analysis'] == 1) & (merged_df['recommend'] == 0),
        (merged_df['sentiment_analysis'] == 0) & (merged_df['recommend'] == 1),
        (merged_df['sentiment_analysis'] == 0) & (merged_df['recommend'] == 0)  
    ],
    scala,
    default=0
)

merged_df


,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5
1,22200,76561197970982479,2,1,Zeno Clash,5
2,227300,js41637,2,1,Euro Truck Simulator 2,5
3,239030,js41637,2,1,"Papers, Please",5
4,370360,evcentric,2,1,TIS-100,5
...,...,...,...,...,...,...
44734,730,76561198239215706,2,1,Counter-Strike: Global Offensive,5
44735,730,wayfeng,1,1,Counter-Strike: Global Offensive,3
44736,253980,76561198251004808,2,1,Enclave,5
44737,730,72947282842,0,1,Counter-Strike: Global Offensive,1


Guardamos el Dataset en un nuevo archivo parquet

In [5]:
merged_df.to_parquet('modelo.parquet')

Leemos el archivo .parquet y lo cargarlo en un DataFrame de pandas

In [6]:
new_df = pd.read_parquet('modelo.parquet')
new_df

,item_id,user_id,sentiment_analysis,recommend,app_name,rating
0,1250,76561197970982479,2,1,Killing Floor,5
1,22200,76561197970982479,2,1,Zeno Clash,5
2,227300,js41637,2,1,Euro Truck Simulator 2,5
3,239030,js41637,2,1,"Papers, Please",5
4,370360,evcentric,2,1,TIS-100,5
...,...,...,...,...,...,...
44734,730,76561198239215706,2,1,Counter-Strike: Global Offensive,5
44735,730,wayfeng,1,1,Counter-Strike: Global Offensive,3
44736,253980,76561198251004808,2,1,Enclave,5
44737,730,72947282842,0,1,Counter-Strike: Global Offensive,1


Adaptar rating_scale a a escala de datos

In [7]:
reader = Reader(rating_scale=(0, 5))  

Cargamos los datos de los usuarios, juegos y calificaciones en un objeto Dataset de Surprise

In [8]:
data = Dataset.load_from_df(new_df[['user_id', 'app_name', 'rating']], reader)

Se divide el conjunto de datos en entrenamiento y prueba

In [9]:
trainset, testset = train_test_split(data, test_size=0.2,random_state=42)  

Optimización de hiperparamétros con GridSeachCV

In [10]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs = -1)
gs.fit(data)

Observamos el RMSE y los hiperparamétros para nuestro modelo 

In [11]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.4896067695799884
{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


Seleccionamos el algoritmo SVD con sus hiperparamétros  y entrenamos el modelo

In [12]:
model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.2)
model.fit(trainset)

Guardamos el modelo en un archivo .pkl

In [13]:
with open('modelo.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

Abrimos el archivo .pkl y lo instanciamos en una en una variable

In [14]:
with open ('modelo.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)

In [15]:
print(modelo)
print("Number of factors:", modelo.n_factors)
print("Number of epochs:", modelo.n_epochs)

Number of factors: 100
Number of epochs: 20


Construyo la función para las recomendaciones 

In [16]:
def recomendacion_usuario( id_usuario ):
    lista=[]
    if id_usuario not in new_df['user_id'].unique():
        return {'error': 'El usuario no existe.'}
    
    #Crear una lista de juegos valorados por el usuario
    juegos_valorados = new_df[new_df['user_id'] == id_usuario]['app_name'].unique()

    #Crear una lista de todos los juegos disponibles
    todos_los_juegos = new_df['app_name'].unique()

    #Crear una lista de juegos no valorados por el usuario específico
    juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))

    #Generar predicciones para los juegos no valorados por el usuario
    predicciones = [modelo.predict(id_usuario, juego) for juego in juegos_no_valorados]

    #Ordenar las predicciones en base a la valoración y obtener los juegos recomendados
    recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5]  # Obtener las 5 mejores recomendaciones

  # Almacenar los nombres de los juegos recomendados en una lista
    juegos_recomendados = [recomendacion.iid for recomendacion in recomendaciones]
    
    # Crear un diccionario con los nombres de los juegos recomendados
    recomendaciones_dict = {
        'Juego 1': juegos_recomendados[0],
        'Juego 2': juegos_recomendados[1],
        'Juego 3': juegos_recomendados[2],
        'Juego 4': juegos_recomendados[3],
        'Juego 5': juegos_recomendados[4]
    }
    
    # Devolver el diccionario con los nombres de los juegos recomendados
    return recomendaciones_dict
    

In [17]:
recomendacion_usuario('js41637')

{'Juego 1': "King Arthur's Gold",
 'Juego 2': 'Gunpoint',
 'Juego 3': 'Psychonauts',
 'Juego 4': 'Yet Another Zombie Defense',
 'Juego 5': 'Dungeon Defenders'}

In [18]:
recomendacion_usuario("maplemage")

{'Juego 1': "King Arthur's Gold",
 'Juego 2': 'Ori and the Blind Forest',
 'Juego 3': 'Dungeon Defenders',
 'Juego 4': 'Rogue Legacy',
 'Juego 5': 'Mark of the Ninja'}

In [19]:
recomendacion_usuario("tecool")

{'error': 'El usuario no existe.'}

In [20]:
recomendacion_usuario("TeCool")

{'Juego 1': "King Arthur's Gold",
 'Juego 2': 'The Wolf Among Us',
 'Juego 3': 'Yet Another Zombie Defense',
 'Juego 4': 'Dungeon Defenders',
 'Juego 5': 'Rogue Legacy'}

: 